# Initialization

In [449]:
import numpy as np
import sympy as sp
from spatialmath import SE3, SO3
from spatialmath.base import r2x

np.set_printoptions(suppress=True, precision=4)

class Robot:
    def __init__(self, params):
        # params is a table of Denavit-Hartenberg convention parameters
        self.params = params
    def A_transform(self, alpha, a, d, theta):
        # generate the Denavit-Hartenberg transformation matrix
        c_theta = np.cos(theta)
        s_theta = np.sin(theta)
        c_alpha = np.cos(alpha)
        s_alpha = np.sin(alpha)

        A_T = np.array([
            [c_theta,    -s_theta * c_alpha,    s_theta * s_alpha,  a * c_theta],
            [s_theta,    c_theta * c_alpha,     -c_theta * s_alpha, a * s_theta],
            [0,          s_alpha,               c_alpha,            d],
            [0,          0,                     0,                  1],
        ])
        return A_T
    def fw_kin(self, joint_angles = None, frame = 6):
        # computes the transformation matrix of the end-effector
        T = np.eye(4)
        for i in range(frame):
            alpha, a, d, theta = self.params[i]
            theta = joint_angles[i]
            T = T @ self.A_transform(alpha, a, d, theta)
            #print("Frame :", i+1, "\n",T)
        T = np.where(np.abs(T) < 1e-10, 0.0, T)
        return T
    def wrist_center(self, R, t):
        # Compute wrist center for inverse kinematics
        d_6 = self.params[5][2]
        z_axis = R[:, 2]
        return t - (d_6 * z_axis)
    def inv_kin(self, R, t):
        # Solve position thetas first
        p_wrist = self.wrist_center(R, t)
        theta_1 = np.arctan2(p_wrist[1], p_wrist[0])
        L_1 = np.sqrt(p_wrist[0]**2 + p_wrist[1]**2)

        a_1 = self.params[0][1]
        a_2 = self.params[1][1]
        d_1 = self.params[0][2]
        a_3 = self.params[2][1]
        d_4 = self.params[3][2]
        
        # ELBOW DOWN ANSWER
        L_2 = np.sqrt((p_wrist[2] - d_1)**2 + (L_1 - a_1)**2)
        L_3 = np.sqrt(a_3**2 + d_4**2)
        beta_1 = np.arctan2(p_wrist[2] - d_1, L_1 - a_1)
        beta_2 = np.arccos((a_2**2 + L_2**2 - L_3**2) / (2 * a_2 * L_2))
        beta_3 = np.arccos((a_2**2 + L_3**2 - L_2**2) / (2 * a_2 * L_3))
        beta_4 = np.arctan2(d_4, a_3)

        ed_theta_2 = beta_1 + beta_2
        ed_theta_3 = (np.pi) - beta_3 - beta_4
        ed_theta_3 = -ed_theta_3

        # IF TIME PERMITS ADD ELBOW UP ANSWER HERE
        eu_theta_2 = beta_2 - beta_1
        eu_theta_2 = -eu_theta_2
        eu_theta_3 = np.pi - beta_3 + beta_4

        # ORIENTATION SOLVER
        # ELBOW DOWN
        ed_theta_123 = [theta_1, ed_theta_2, ed_theta_3]

        ed_R_03 = SE3(self.fw_kin(ed_theta_123, 3))
        ed_R_36 = ed_R_03.R.T @ R
    
        ed_theta_456 = np.array(r2x(ed_R_36, representation='eul'))

        # ELBOW UP
        eu_theta_123 = [theta_1, eu_theta_2, eu_theta_3]
        eu_R_03 = SE3(self.fw_kin(eu_theta_123, 3))
        eu_R_36 = eu_R_03.R.T @ R
    
        eu_theta_456 = np.array(r2x(eu_R_36, representation='eul'))


        q_ed = np.array([theta_1, ed_theta_2, ed_theta_3])
        q_ed = np.append(q_ed, ed_theta_456)

        q_eu = np.array([theta_1, eu_theta_2, eu_theta_3])
        q_eu = np.append(q_eu, eu_theta_456)

        return q_ed, q_eu
    
    def compute_jacobian(self,joint_angles = None, frame = 6):
        # (previous columns z axis) cross (position vector of current frame - previous frames position vector)
        initial = SE3(np.eye(4))
        J_v = np.empty((3, 0))
        J_omega = np.empty((3, 0))
        T_end = SE3(self.fw_kin(joint_angles, 6))
        t_end = T_end.t
        for i in range(frame):

            if i == 0:
                z_previous = initial.R[:, 2]
                t_previous = initial.t
            else:
                T_previous = SE3(self.fw_kin(joint_angles, i))
                z_previous = T_previous.R[:, 2]
                t_previous = T_previous.t

            cross = np.cross(z_previous, (t_end - t_previous)).reshape(3,-1)
            J_v = np.hstack((J_v, cross))
            J_omega = np.hstack((J_omega, z_previous.reshape(3, -1)))

        J = np.vstack((J_v, J_omega))
        return J








# Problem 1A Forward Kinematics

In [450]:

# *******************************************
#       DEFINE DENAVIT-HARTENBERG TABLE
# *******************************************
dh_table = [
    (np.pi/2,     27.5,   339.0,  0.0      ),     # Joint 1
    (0.0,         250.0,  0.0,    0.0      ),     # Joint 2
    (np.pi/2,     70.0,   0.0,    0.0      ),     # Joint 3
    (-np.pi/2,    0.0,    250.0,  0.0      ),     # Joint 4
    (np.pi/2,     0.0,    0.0,    0.0      ),     # Joint 5
    (0.0,         0.0,    95.0,   0.0      )      # Joint_6
]


# Set various joint positions to forward kinematics
home = [0.0, np.pi/2, 0.0, 0.0, 0.0, 0.0]
problem_1 = [0.0, np.pi/2, 0.0, 0.0, -np.pi/2, 0.0]

# Initiate robot
minibot = Robot(dh_table)

# Home Position Forward Kinematics
end_effector = minibot.fw_kin(home,)
end_effector = SE3(end_effector)
print("Home Position\n", end_effector)

# Problem 1A Forward Kinematics
end_effector = minibot.fw_kin(problem_1)
end_effector = SE3(end_effector)


position_vector = end_effector.t
rpy_vector = np.rad2deg(r2x(end_effector.R, representation='rpy/zyx'))
rpy_vector = rpy_vector[::-1]
vector = np.append(position_vector, rpy_vector)
print(end_effector.R)
print("Problem 1 Position\n", end_effector)
print("Problem 1 Vector format (t, Euler ZYX)\n", vector)

Home Position
    0         0         1         372.5     
   0        -1         0         0         
   1         0         0         659       
   0         0         0         1         

[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
Problem 1 Position
    1         0         0         277.5     
   0        -1         0         0         
   0         0        -1         564       
   0         0         0         1         

Problem 1 Vector format (t, Euler ZYX)
 [277.5   0.  564.    0.   -0.  180. ]


# Problem 1B Inverse Kinematics

In [451]:
# Problem initial rotation and position vector

R_init = np.array([
    [0.7551, 0.4013, 0.5184],
    [0.6084, -0.7235, -0.3262],
    [0.2441, 0.5617, -0.7905]
])

t_init = np.array([399.1255, 171.01529, 416.0308])


# Inverse Kinematics call
inv1, inv2 = minibot.inv_kin(R_init, t_init)

#Test Solution 1
print("**************Solution 1**************")
print("**************Elbow Down**************")
print("Theta's 1-6 (radian)\n", inv1)
print("Theta's 1-6 (degree)\n", np.rad2deg(inv1).round(2))
test1 = SE3(minibot.fw_kin(inv1, 6))
print("Test inverse to forward kin: \n", test1)


#Test SOlution 2
print("**************Solution 2**************")
print("************** Elbow Up **************")
print("Theta's 1-6 (radian)\n", inv2)
print("Theta's 1-6 (degree)\n", np.rad2deg(inv2).round(2))
test2 = SE3(minibot.fw_kin(inv2, 6))
print("Test inverse to forward kin: \n", test2)

#Print given for comparison
print("Given: \n", SE3.Rt((R_init),t_init, check = False))

**************Solution 1**************
**************Elbow Down**************
Theta's 1-6 (radian)
 [ 0.5236  1.0472 -0.      2.3562  0.8727 -2.7489]
Theta's 1-6 (degree)
 [  30.    60.    -0.   135.    50.  -157.5]
Test inverse to forward kin: 
    0.7551    0.4013    0.5184    399.1     
   0.6084   -0.7235   -0.3262    171       
   0.2441    0.5617   -0.7905    416       
   0         0         0         1         

**************Solution 2**************
************** Elbow Up **************
Theta's 1-6 (radian)
 [ 0.5236 -0.2792  2.5956  2.5314  1.9031  2.7387]
Theta's 1-6 (degree)
 [ 30.   -16.   148.72 145.04 109.04 156.92]
Test inverse to forward kin: 
    0.7551    0.4013    0.5184    399.1     
   0.6084   -0.7235   -0.3262    171       
   0.2441    0.5617   -0.7905    416       
   0         0         0         1         

Given: 
    0.7551    0.4013    0.5184    399.1     
   0.6084   -0.7235   -0.3262    171       
   0.2441    0.5617   -0.7905    416       
   0       

# Problem 2 Jacobian

In [452]:
# Calculate Linear Jacobean Terms
# (previous columns z axis) cross (position vector of current frame - previous frames position vector)
v = np.array([100, -200, -300]).reshape(3, 1)
omega = np.array([2, -1, 0.5]).reshape(3, 1)
twist = np.vstack((v, omega))  # (6x1)
print(twist.shape)

jacobian = minibot.compute_jacobian(inv1, 6)
print(jacobian)
q_dot = np.linalg.pinv(jacobian) @ twist

testing = jacobian @ q_dot

#print(testing)
print(q_dot)

(6, 1)
[[-171.0158  -66.7116  120.7885  -48.0121  -51.6866    0.    ]
 [ 399.1246  -38.5164   69.7381   31.6982  -79.7025   -0.    ]
 [   0.      403.6597  278.661   -44.566    -1.0062    0.    ]
 [   0.        0.5       0.5       0.75     -0.6597    0.5184]
 [   0.       -0.866    -0.866     0.433     0.4356   -0.3262]
 [   1.        0.        0.       -0.5      -0.6124   -0.7905]]
[[-0.8151]
 [ 0.4255]
 [-1.7791]
 [-0.4597]
 [-3.5176]
 [ 1.3522]]


$$
\mathbf{A}_i =
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
$$